In [1]:
import math
import random
from collections import defaultdict
import time
from itertools import permutations # To get all permutations of A and B

class M3System:
    def __init__(self, A, B, C, D, E, modulus):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        self.E = E
        self.modulus = modulus
        self.identity_element = M3Element(0, 0, 0, self) 

    def __str__(self):
        return f"M3System(A={self.A}, B={self.B}, C={self.C}, D={self.D}, E={self.E}, modulus={self.modulus})"

    def get_identity(self):
        return self.identity_element

class M3Element:
    def __init__(self, x0, x1, x2, system):
        self.x0 = x0 % system.modulus
        self.x1 = x1 % system.modulus
        self.x2 = x2 % system.modulus
        self.system = system

    def __mul__(self, other):
        if not isinstance(other, M3Element) or self.system != other.system:
            raise ValueError("Can only multiply M3Element with another M3Element from the same system.")

        s = self.system
        new_x0 = (self.x0 + other.x0 + self.x0 * other.x0 +
                  s.A * self.x1 * other.x1 +
                  s.C * self.x2 * other.x1 +
                  s.B * self.x2 * other.x2) % s.modulus

        new_x1 = (self.x1 + other.x1 + self.x1 * other.x0 +
                  self.x0 * other.x1 +
                  s.D * self.x1 * other.x1 +
                  s.E * self.x1 * other.x2) % s.modulus

        new_x2 = (self.x2 + other.x2 + self.x2 * other.x0 +
                  self.x0 * other.x2 +
                  s.D * self.x2 * other.x1 +
                  s.E * self.x2 * other.x2) % s.modulus

        return M3Element(new_x0, new_x1, new_x2, s)

    def __pow__(self, exponent):
        if not isinstance(exponent, int) or exponent < 0:
            raise ValueError("Exponent must be a non-negative integer.")
        if exponent == 0:
            return self.system.get_identity()
        
        result = self.system.get_identity() 
        base = self

        while exponent > 0:
            if exponent % 2 == 1:
                result = result * base
            base = base * base
            exponent //= 2
        return result

    def __eq__(self, other):
        if not isinstance(other, M3Element):
            return NotImplemented
        return (self.x0 == other.x0 and
                self.x1 == other.x1 and
                self.x2 == other.x2 and
                self.system == other.system)

    def __hash__(self):
        return hash((self.x0, self.x1, self.x2, hash(self.system)))

    def __str__(self):
        return f"[{self.x0}, {self.x1}, {self.x2}]"

    def __repr__(self):
        return self.__str__()

    def to_tuple(self):
        return (self.x0, self.x1, self.x2)

def is_prime(n):
    """Checks if a number is prime."""
    if n < 2: return False
    if n == 2 or n == 3: return True
    if n % 2 == 0 or n % 3 == 0: return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def get_primes_in_range(start, end):
    """Returns a list of prime numbers in a given range."""
    return [n for n in range(start, end + 1) if is_prime(n)]

def analyze_all_orbits(system):
    """
    Analyzes all possible orbits within the given M3System.
    It iterates through all elements in the system's space,
    finds their orbit lengths, and categorizes them.
    """
    modulus = system.modulus
    total_space_elements = modulus**3
    
    generated_cycles_lengths = []
    global_elements_in_orbits = set() # Keeps track of all elements visited across all orbits

    identity_element = system.get_identity()
    identity_tuple = identity_element.to_tuple()
    
    # The identity element forms a cycle of length 1 by itself
    if identity_tuple not in global_elements_in_orbits:
        global_elements_in_orbits.add(identity_tuple)
        generated_cycles_lengths.append(1)

    # Iterate through all possible elements in the system's space
    for x0_val in range(modulus):
        for x1_val in range(modulus):
            for x2_val in range(modulus):
                current_seed = M3Element(x0_val, x1_val, x2_val, system)
                current_seed_tuple = current_seed.to_tuple()

                # If this element has already been visited as part of another orbit, skip it
                if current_seed_tuple in global_elements_in_orbits:
                    continue
                
                path = [] # Stores the sequence of elements in the current path
                path_dict = {} # Maps element tuples to their index in the path to detect cycles
                current_in_orbit = current_seed
                
                cycle_length = 0
                
                while True:
                    current_tuple = current_in_orbit.to_tuple()
                    
                    # If the current element is already in the path, a cycle has been found
                    if current_tuple in path_dict:
                        cycle_start_index = path_dict[current_tuple]
                        cycle_length = len(path) - cycle_start_index
                        break
                    
                    # If the current element is the identity element, it's a special cycle ending at identity
                    if current_in_orbit == identity_element:
                        path.append(current_tuple)
                        cycle_length = len(path)
                        global_elements_in_orbits.add(identity_tuple) # Ensure identity is marked as visited
                        break
                    
                    path_dict[current_tuple] = len(path)
                    path.append(current_tuple)
                    
                    # Calculate the next element in the orbit
                    current_in_orbit = current_in_orbit * current_seed

                generated_cycles_lengths.append(cycle_length)
                
                # Add all elements in the found path/cycle to the global set of visited elements
                for p_elem_tuple in path:
                    global_elements_in_orbits.add(p_elem_tuple)
    
    total_found_orbits = len(generated_cycles_lengths)
    
    N_minus_1_count = 0
    N_sq_minus_1_count = 0
    N_minus_1_half_count = 0
    N_sq_minus_1_half_count = 0

    target_N_minus_1 = modulus - 1
    target_N_sq_minus_1 = modulus * modulus - 1
    target_N_minus_1_half = (modulus - 1) // 2 if (modulus - 1) % 2 == 0 else None
    target_N_sq_minus_1_half = (modulus * modulus - 1) // 2 if (modulus * modulus - 1) % 2 == 0 else None

    # Count how many orbits match specific target lengths
    for length in generated_cycles_lengths:
        if length == target_N_minus_1:
            N_minus_1_count += 1
        elif length == target_N_sq_minus_1:
            N_sq_minus_1_count += 1
        elif target_N_minus_1_half is not None and length == target_N_minus_1_half:
            N_minus_1_half_count += 1
        elif target_N_sq_minus_1_half is not None and length == target_N_sq_minus_1_half:
            N_sq_minus_1_half_count += 1
    
    # Calculate shares of orbits with specific lengths
    if total_found_orbits > 0:
        share_N_minus_1 = N_minus_1_count / total_found_orbits
        share_N_sq_minus_1 = N_sq_minus_1_count / total_found_orbits
        share_N_minus_1_half = N_minus_1_half_count / total_found_orbits
        share_N_sq_minus_1_half = N_sq_minus_1_half_count / total_found_orbits
    else:
        share_N_minus_1 = share_N_sq_minus_1 = share_N_minus_1_half = share_N_sq_minus_1_half = 0

    return {
        "Modulus": modulus,
        "A": system.A, "B": system.B, "C": system.C, "D": system.D, "E": system.E,
        "Total_Space_Elements": total_space_elements,
        "Total_Unique_Elements_Visited": len(global_elements_in_orbits),
        "Total_Orbits_Found": total_found_orbits,
        f"Orbits_Length_N-1 ({target_N_minus_1})_Count": N_minus_1_count,
        f"Share_N-1": share_N_minus_1,
        f"Orbits_Length_N^2-1 ({target_N_sq_minus_1})_Count": N_sq_minus_1_count,
        f"Share_N^2-1": share_N_sq_minus_1,
        f"Orbits_Length_N-1/2 ({target_N_minus_1_half})_Count": N_minus_1_half_count if target_N_minus_1_half is not None else 0,
        f"Share_N-1/2": share_N_minus_1_half,
        f"Orbits_Length_N^2-1/2 ({target_N_sq_minus_1_half})_Count": N_sq_minus_1_half_count if target_N_sq_minus_1_half is not None else 0,
        f"Share_N^2-1/2": share_N_sq_minus_1_half,
    }


if __name__ == "__main__":
    test_modulus = 23 # You can change this to 11, 13 etc.
                     # Be mindful of runtime for larger N, as it's an exhaustive test for A, B.

    # Fixed values for C, D, E
    FIXED_C = 1
    FIXED_D = 1
    FIXED_E = 2

    # Ensure fixed values are valid (non-zero and less than modulus)
    if not (0 <= FIXED_C < test_modulus and
            0 <= FIXED_D < test_modulus and
            0 <= FIXED_E < test_modulus):
        raise ValueError(
            f"Fixed C, D, E values (1) are not valid for modulus {test_modulus}. "
            "They must be non-zero and less than N."
        )

    # Generate all possible distinct non-zero values for A and B
    # excluding the fixed C, D, E if they happen to be 1.
    available_values_for_AB = list(range(1, test_modulus))
    
    # Need at least 2 unique values for A and B to form permutations
    if len(available_values_for_AB) < 2:
        raise ValueError(
            f"Not enough distinct non-zero values available for A and B."
            f"Available: {available_values_for_AB}"
        )
            
    # Generate all permutations of 2 distinct values for A and B
    param_combinations = list(permutations(available_values_for_AB, 2))
    total_combinations = len(param_combinations)

    print(f"Starting exhaustive test for modulus N = {test_modulus}")
    print(f"C, D, E fixed at: C={FIXED_C}, D={FIXED_D}, E={FIXED_E}")
    print(f"----------------------------------------------------------------------------------------------------")
    print(f"Total parameter combinations (A, B) for N={test_modulus}: {total_combinations}")

    start_total_time = time.time()
    
    # Statistics collectors
    min_orbits = float('inf')
    max_orbits = float('-inf')
    sum_orbits = 0
    orbit_counts = [] # For median
    
    # Share distribution data
    share_N_sq_minus_1_values = []
    share_N_minus_1_values = []
    share_N_sq_minus_1_half_values = []
    share_N_minus_1_half_values = []

    # Store parameters for max/min shares
    max_share_N_sq_minus_1 = -1.0
    params_max_N_sq_minus_1 = None
    min_share_N_sq_minus_1 = 1.1 # Greater than any possible share (shares are 0.0 to 1.0)
    params_min_N_sq_minus_1 = None
    max_share_N_minus_1 = -1.0
    params_max_N_minus_1 = None

    for i, (A, B) in enumerate(param_combinations):
        current_iter_start_time = time.time()
        
        system = M3System(A, B, FIXED_C, FIXED_D, FIXED_E, test_modulus)
        
        # print(f"  Test {i+1}/{total_combinations} with parameters: A={A}, B={B}, C={FIXED_C}, D={FIXED_D}, E={FIXED_E}")
        result = analyze_all_orbits(system)
        
        end_iter_time = time.time()
        time_taken_iter = end_iter_time - current_iter_start_time

        # Update statistics
        total_orbits = result["Total_Orbits_Found"]
        min_orbits = min(min_orbits, total_orbits)
        max_orbits = max(max_orbits, total_orbits)
        sum_orbits += total_orbits
        orbit_counts.append(total_orbits)

        share_N_sq_minus_1_values.append(result[f'Share_N^2-1'])
        share_N_minus_1_values.append(result[f'Share_N-1'])
        share_N_sq_minus_1_half_values.append(result[f'Share_N^2-1/2'])
        share_N_minus_1_half_values.append(result[f'Share_N-1/2'])

        # Update max/min share parameters
        if result[f'Share_N^2-1'] > max_share_N_sq_minus_1:
            max_share_N_sq_minus_1 = result[f'Share_N^2-1']
            params_max_N_sq_minus_1 = (A, B, FIXED_C, FIXED_D, FIXED_E)
        if result[f'Share_N^2-1'] < min_share_N_sq_minus_1:
            min_share_N_sq_minus_1 = result[f'Share_N^2-1']
            params_min_N_sq_minus_1 = (A, B, FIXED_C, FIXED_D, FIXED_E)
        if result[f'Share_N-1'] > max_share_N_minus_1:
            max_share_N_minus_1 = result[f'Share_N-1']
            params_max_N_minus_1 = (A, B, FIXED_C, FIXED_D, FIXED_E)

        # Progress update
        if (i + 1) % 100 == 0 or (i + 1) == total_combinations:
            print(f"  Progress: {i+1}/{total_combinations} combinations completed. ({time_taken_iter:.2f} sec)")

    end_total_time = time.time()
    total_execution_time = end_total_time - start_total_time

    # Final summary output
    print(f"\n====================================================================================================")
    print(f"=== EXHAUSTIVE ANALYSIS FOR N = {test_modulus} ===")
    print(f"Total parameter combinations (A, B) tested: {total_combinations}")
    print(f"Total execution time: {total_execution_time:.2f} seconds.")
    print(f"====================================================================================================")

    # Calculate median orbits
    orbit_counts.sort()
    mid = len(orbit_counts) // 2
    median_orbits = (orbit_counts[mid - 1] + orbit_counts[mid]) / 2 if len(orbit_counts) % 2 == 0 else orbit_counts[mid]

    print("\n--- Distribution of Number of Orbits Found ---")
    print(f"  Minimum: {min_orbits:.4f}")
    print(f"  Maximum: {max_orbits:.4f}")
    print(f"  Average: {sum_orbits / total_combinations:.4f}")
    print(f"  Median: {median_orbits:.4f}")

    # Calculate statistics for shares
    print("\n--- Distribution of Share of Orbits with Length N^2-1 ---")
    print(f"  Minimum: {min(share_N_sq_minus_1_values):.4f}")
    print(f"  Maximum: {max(share_N_sq_minus_1_values):.4f}")
    print(f"  Average: {sum(share_N_sq_minus_1_values) / total_combinations:.4f}")
    share_N_sq_minus_1_values.sort()
    mid = len(share_N_sq_minus_1_values) // 2
    median_N_sq_minus_1 = (share_N_sq_minus_1_values[mid - 1] + share_N_sq_minus_1_values[mid]) / 2 if len(share_N_sq_minus_1_values) % 2 == 0 else share_N_sq_minus_1_values[mid]
    print(f"  Median: {median_N_sq_minus_1:.4f}")

    print("\n--- Distribution of Share of Orbits with Length N-1 ---")
    print(f"  Minimum: {min(share_N_minus_1_values):.4f}")
    print(f"  Maximum: {max(share_N_minus_1_values):.4f}")
    print(f"  Average: {sum(share_N_minus_1_values) / total_combinations:.4f}")
    share_N_minus_1_values.sort()
    mid = len(share_N_minus_1_values) // 2
    median_N_minus_1 = (share_N_minus_1_values[mid - 1] + share_N_minus_1_values[mid]) / 2 if len(share_N_minus_1_values) % 2 == 0 else share_N_minus_1_values[mid]
    print(f"  Median: {median_N_minus_1:.4f}")

    print("\n--- Distribution of Share of Orbits with Length (N^2-1)/2 ---")
    print(f"  Minimum: {min(share_N_sq_minus_1_half_values):.4f}")
    print(f"  Maximum: {max(share_N_sq_minus_1_half_values):.4f}")
    print(f"  Average: {sum(share_N_sq_minus_1_half_values) / total_combinations:.4f}")
    share_N_sq_minus_1_half_values.sort()
    mid = len(share_N_sq_minus_1_half_values) // 2
    median_N_sq_minus_1_half = (share_N_sq_minus_1_half_values[mid - 1] + share_N_sq_minus_1_half_values[mid]) / 2 if len(share_N_sq_minus_1_half_values) % 2 == 0 else share_N_sq_minus_1_half_values[mid]
    print(f"  Median: {median_N_sq_minus_1_half:.4f}")

    print("\n--- Distribution of Share of Orbits with Length (N-1)/2 ---")
    print(f"  Minimum: {min(share_N_minus_1_half_values):.4f}")
    print(f"  Maximum: {max(share_N_minus_1_half_values):.4f}")
    print(f"  Average: {sum(share_N_minus_1_half_values) / total_combinations:.4f}")
    share_N_minus_1_half_values.sort()
    mid = len(share_N_minus_1_half_values) // 2
    median_N_minus_1_half = (share_N_minus_1_half_values[mid - 1] + share_N_minus_1_half_values[mid]) / 2 if len(share_N_minus_1_half_values) % 2 == 0 else share_N_minus_1_half_values[mid]
    print(f"  Median: {median_N_minus_1_half:.4f}")

    print("\n--- Example Parameters ---")
    if params_max_N_sq_minus_1:
        print(f"Parameters with max share of N^2-1 orbits ({max_share_N_sq_minus_1:.4f}): A,B,C,D,E = {params_max_N_sq_minus_1}")
    if params_min_N_sq_minus_1:
        print(f"Parameters with min share of N^2-1 orbits ({min_share_N_sq_minus_1:.4f}): A,B,C,D,E = {params_min_N_sq_minus_1}")
    if params_max_N_minus_1:
        print(f"Parameters with max share of N-1 orbits ({max_share_N_minus_1:.4f}): A,B,C,D,E = {params_max_N_minus_1}")

Starting exhaustive test for modulus N = 23
C, D, E fixed at: C=1, D=1, E=2
----------------------------------------------------------------------------------------------------
Total parameter combinations (A, B) for N=23: 462
  Progress: 100/462 combinations completed. (0.02 sec)
  Progress: 200/462 combinations completed. (0.02 sec)
  Progress: 300/462 combinations completed. (0.02 sec)
  Progress: 400/462 combinations completed. (0.02 sec)
  Progress: 462/462 combinations completed. (0.02 sec)

=== EXHAUSTIVE ANALYSIS FOR N = 23 ===
Total parameter combinations (A, B) tested: 462
Total execution time: 9.19 seconds.

--- Distribution of Number of Orbits Found ---
  Minimum: 70.0000
  Maximum: 975.0000
  Average: 520.5606
  Median: 522.0000

--- Distribution of Share of Orbits with Length N^2-1 ---
  Minimum: 0.0000
  Maximum: 0.3286
  Average: 0.0277
  Median: 0.0214

--- Distribution of Share of Orbits with Length N-1 ---
  Minimum: 0.0000
  Maximum: 0.8921
  Average: 0.7963
  Media